In [1]:
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image

from network.mynetwork_uu import Unet
from loss.loss import CLIPLoss
from utils.func import get_features,vgg_normalize

import time

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
print(torch.cuda.is_available())

cuda
True


In [3]:
model = Unet(device).to(device)
# model = Unet().to(device)
cliploss = CLIPLoss(device)
mseloss = torch.nn.MSELoss()
# vgg = torchvision.models.vgg19(pretrained=True).features.to(device)
# for x in vgg.parameters():
#     x.requires_grad = False

topil = transforms.ToPILImage()
topic = transforms.ToTensor()

In [4]:
lr1 = 0.001
lr2 = 0.0003

dir_lambda = 500
content_lambda = 150
patch_lambda = 9000
norm_lambda = 0.002
gol_lambda = 300


In [5]:
loss_li = [0]*100

In [6]:
pil = Image.open(f"./source_pic/people0.jpg")
# pil = Image.open(f"resulto.jpg")
ori_size = pil.size[::-1]
pil = transforms.Resize(size=(512, 512), interpolation=Image.BICUBIC)(pil)
pic = topic(pil).unsqueeze(0).to(device)
# pic = torch.ones(1, 3, 512, 512).to(device)
pic.requires_grad = False
# pic[:,1,:,:] = 0

C:\Users\win10\AppData\Local\Temp\ipykernel_29996\3599116407.py:4: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  pil = transforms.Resize(size=(512, 512), interpolation=Image.BICUBIC)(pil)
E:\Anaconda\envs\sth\lib\site-packages\torchvision\transforms\transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [7]:
path = "result/result.jpg"
# path = "0.jpg"

In [8]:
big11 = torch.nn.Upsample(scale_factor=8, mode='bilinear', align_corners=True)
big22 = torch.nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)

list_selected = [4,9,14,19,39,59,79,99]

# hot
# hot_list = []

# box
patch_centers = []
patch_sizes = []
pic_patched = []

In [9]:
# source = "photo"
# source = "cat"
# source = "CG picture"
source = "unhappy"
# source = "man"
# source = "flower"
# source = "blue sky"
# source = "brown hair"

# target = "man with thick black moustache"
# target = "black sky"
# target = "sketch with pencil"
# target = "dark black hair"
# target = "gold boat"
# target = "comic art"
# target = "Barack Obama"
# target = "joker"
# target = "Mark Elliot Zuckerberg"
# target = "Mona Lisa by Leonardo da Vinci"
# target = "angry"
target = "very happy"
# target = "girl"
# target = "Neon Light"
# target = "shining gold car"
# target = "pop art"
# target = "old man"
# target = "Watercolor Art with Thick Brush"
# target = "Pixar"
# target = "watercolor painting with thick brush"
# target = "wheat field by Van Gogh"
# target = "starry night by Van Gogh"
# target = "the scream by Edvard Munch"
# target = "Monet"
# target = "the girl with a pearl earring by Vermeer"
# target = "painting by Paul Gauguin"
# target = "oil painting by Raffaello Santi"
# target = "poster in 1940s"
# target = "oil painting by renoir"
# target = "Barbizon School"
# target = "steampunk"
# target = "oil painting with thick brush"
# target = "woman Wearing Raybon Sunglasses"
# target = "the great wave off kanagawa"
# target = "mosaic"
# target = "fire"
# target = "Chinese Brush Painting of mountains in black and white"
# target = "snowy"
# target = "sketch with pencil"
# target = "pop art of night city"
# target = "cubism"
# target = "Cartoon"
# target = "anime"
# target = "white marble carving"
# target = "Self-portrait by Pablo Picasso "
# target = "David Hockney"


In [10]:
dli = []
nli = []

In [11]:
# patch.eps
# patch_file = open(f"./data/nums.txt", mode="w")

In [12]:
def train(iteration1, iteration2, pic, source, target):


    input = pic

    # pic_f = get_features(vgg_normalize(pic), vgg)
    # print(model.parameters())
    opt = optim.Adam(model.parameters(), lr=lr1)
    for i in range(iteration1):
        opt.zero_grad()
        neo_pic = model(input)
        loss = mseloss(pic, neo_pic) * 1

        # loss = 0
        # neo_pic_f = get_features(vgg_normalize(neo_pic), vgg)
        # loss += torch.mean((pic_f['conv4_2'] - neo_pic_f['conv4_2']) ** 2)
        # loss += torch.mean((pic_f['conv5_2'] - neo_pic_f['conv5_2']) ** 2)

        loss.backward()
        opt.step()

        with torch.no_grad():
            print("iter:", i + 1, "loss:", loss.item())

    with torch.no_grad():
        neo_pic = model(input)
        pil = topil(neo_pic.squeeze(0).cpu())
        pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)
        pil.save(f"mid.jpg")

    # pic_f = get_features(vgg_normalize(pic),vgg)

    opt = optim.Adam(model.parameters(), lr=lr2)

    for i in range(iteration2):


        opt.zero_grad()

        neo_pic = model(input)

        dir_loss = 0
        dir_loss += cliploss.forward_dir(pic, source, neo_pic, target)

        gol_loss = 0
        # gol_loss += cliploss.forward_gol(pic, source, neo_pic, target)

        content_loss = 0
        # content_loss += mseloss(pic, neo_pic)
        # neo_pic_f = get_features(vgg_normalize(neo_pic), vgg)
        # content_loss += torch.mean((pic_f['conv4_2'] - neo_pic_f['conv4_2']) ** 2)
        # content_loss += torch.mean((pic_f['conv5_2'] - neo_pic_f['conv5_2']) ** 2)

        patch_loss = 0
        # patch_loss += cliploss.forward_patch(pic, source, neo_pic, target)

        norm_loss = 0
        norm_loss += cliploss.forward_prior(pic, source, neo_pic, target)

        loss = dir_loss * dir_lambda + \
               content_loss * content_lambda + \
               patch_loss * patch_lambda + \
               norm_loss * norm_lambda + \
               gol_loss * gol_lambda


        patch_loss_fast,patch_loss_slow, li = cliploss.forward_patch_sec(pic, source, neo_pic, target)
        patch_loss_fast *= patch_lambda
        patch_loss_slow *= patch_lambda

        for x in model.res2.parameters():
            x.requires_grad = False
        patch_loss_slow.backward(retain_graph=True)
        for x in model.res2.parameters():
            x.requires_grad = True

        for x in model.res.parameters():
            x.requires_grad = False
        for x in model.conv3.parameters():
            x.requires_grad = False
        for x in model.upsample3.parameters():
            x.requires_grad = False
        for x in model.deconv3.parameters():
            x.requires_grad = False
        patch_loss_fast.backward(retain_graph=True)
        for x in model.res.parameters():
            x.requires_grad = True
        for x in model.conv3.parameters():
            x.requires_grad = True
        for x in model.upsample3.parameters():
            x.requires_grad = True
        for x in model.deconv3.parameters():
            x.requires_grad = True

        loss.backward()
        # (loss+patch_loss_fast+patch_loss_slow).backward()


        opt.step()

        # opt_fast.step()
        # opt_slow.step()

        with torch.no_grad():
            loss_li[i]+=(loss+patch_loss_fast+patch_loss_slow).item()
            print("iter:", i + 1, "fast_loss:", patch_loss_fast.item(), "slow_loss:", patch_loss_slow.item())
            print("iter:", i + 1, "loss:", (loss+patch_loss_fast+patch_loss_slow).item())


            # patch.eps

            # for x in li:
            #     patch_file.write(str(x)+" ")
            #
            # patch_file.write("\n")

            if (i+1)%10==0  or i==0:
                neo_pic = model(input)
                pil = topil(neo_pic.squeeze(0).cpu())
                pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)
                pil.save(f"mid/{i//10}.jpg")

            # show loss

            # dli.append(dir_loss.item()*dir_lambda)
            # nli.append(norm_loss.item()*norm_lambda)




            if i in list_selected:

                # hot

                # a1 = model.res[0].block[0].spatialattention.hot.detach()
                # a2 = model.res[2].block[0].spatialattention.hot.detach()
                # a3 = model.res2[0].block[0].spatialattention.hot.detach()
                # a4 = model.res2[2].block[0].spatialattention.hot.detach()
                #
                # hot1 = big11(a1)[0,0,:,:]
                # hot2 = big11(a2)[0,0,:,:]
                # hot3 = big22(a3)[0,0,:,:]
                # hot4 = big22(a4)[0,0,:,:]
                #
                # hot = hot1+hot2+hot3+hot4
                #
                # hot = hot.cpu().numpy()
                #
                # hot_list.append(hot)


                # box
                patch_centers.append(cliploss.patch_points.tolist())
                patch_sizes.append(cliploss.patch_size)

                pic_patched.append(model(input))

                pass




    # return  model(input)
    # neo_pic = model(input)
    # pil = topil(neo_pic.squeeze(0).cpu())
    # # pil.save(f"{source}-{target}.jpg")
    # pil.save(path)





In [13]:
start = time.time()
train(100, 100, pic, source, target)
end = time.time()
usetime = end - start
print(f"usetime: {usetime}")

iter: 1 loss: 0.04537791758775711
iter: 2 loss: 0.022188326343894005
iter: 3 loss: 0.012859868817031384
iter: 4 loss: 0.00797811895608902
iter: 5 loss: 0.005427561234682798
iter: 6 loss: 0.0039007775485515594
iter: 7 loss: 0.002926961984485388
iter: 8 loss: 0.002283369190990925
iter: 9 loss: 0.001848921412602067
iter: 10 loss: 0.001539385411888361
iter: 11 loss: 0.0013066524406895041
iter: 12 loss: 0.0011257764417678118
iter: 13 loss: 0.0009918028954416513
iter: 14 loss: 0.0009054010733962059
iter: 15 loss: 0.0008487081504426897
iter: 16 loss: 0.0008043148554861546
iter: 17 loss: 0.0007690989878028631
iter: 18 loss: 0.0007501982618123293
iter: 19 loss: 0.000747989397495985
iter: 20 loss: 0.000748860533349216
iter: 21 loss: 0.0007403648342005908
iter: 22 loss: 0.0007229816983453929
iter: 23 loss: 0.0007002974743954837
iter: 24 loss: 0.0006697738426737487
iter: 25 loss: 0.0006320439861156046
iter: 26 loss: 0.000596296158619225
iter: 27 loss: 0.000568876916076988
iter: 28 loss: 0.00054755

C:\Users\win10\AppData\Local\Temp\ipykernel_29996\1047062930.py:28: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)
C:\Users\win10\AppData\Local\Temp\ipykernel_29996\1047062930.py:128: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)

KeyboardInterrupt



In [ ]:
neo_pic = model(pic)
pil = topil(neo_pic.squeeze(0).cpu())
pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)
pil.save(path)


In [ ]:
# patch.eps
# patch_file.close()

In [ ]:
# file_uu = open("neo_uu.txt", "r")
# read_loss_li = file_uu.readline()
# read_loss_li = [float(x) for x in read_loss_li.split()]
# if not read_loss_li:
#     read_loss_li = [0]*100
# cur_times = int(file_uu.readline())
# file_uu.close()

In [ ]:
# import matplotlib.pyplot as plt
# x = [i for i in range(100)]
# plt.plot(x,loss_li[:100],color="red",marker="o")



In [ ]:
# for i in range(100):
#     loss_li[i]+=read_loss_li[i]
# with open(file = "neo_uu.txt", mode = "w") as file:
#     for i in loss_li:
#         file.write(str(i)+" ")
#     file.write("\n")
#     file.write(str(cur_times+1))


In [ ]:
# x = [i for i in range(100)]
# plt.plot(x,gli,color="red",marker="o")

In [ ]:
# show loss

# x = [i for i in range(100)]
# plt.plot(x,dli,color="red",marker="o")

In [ ]:
# show loss

# x = [i for i in range(100)]
# plt.plot(x,nli,color="red",marker="o")

In [ ]:
# show loss

# x = [i for i in range(100)]
# li = [dli[i]+nli[i] for i in range(100)]
# plt.plot(x,li,color="red",marker="o")

In [ ]:
# show loss

# x = [i for i in range(100)]
# li = [x/64 for x in cliploss.right_patch]
# plt.plot(x,li,color="red",marker="o")

In [ ]:
# show loss

# print(sum(cliploss.right_patch)/6400)

In [ ]:
# show hot

# i = 0
# for hot in hot_list:
#     fig,_ = plt.subplots()
#     plt.imshow(hot)
#     plt.tight_layout()
#     plt.show()
#     fig.savefig(f'./hot/{i}.eps',dpi=600,format='eps')
#     i+=1

In [ ]:
# box

# for j in range(8):
#     pic = pic_patched[j]
#     size = patch_sizes[j]
#     points = patch_centers[j]
#
#     hsize = size//2
#
#     for y,x in points:
#         for i in range(-hsize, hsize+1):
#             pic[0][0][y+i][x+hsize] = 1
#             pic[0][0][y+i][x-hsize] = 1
#             pic[0][0][y+hsize][x+i] = 1
#             pic[0][0][y-hsize][x+i] = 1
#
#             pic[0][1][y+i][x+hsize] = 0
#             pic[0][1][y+i][x-hsize] = 0
#             pic[0][1][y+hsize][x+i] = 0
#             pic[0][1][y-hsize][x+i] = 0
#
#             pic[0][2][y+i][x+hsize] = 0
#             pic[0][2][y+i][x-hsize] = 0
#             pic[0][2][y+hsize][x+i] = 0
#             pic[0][2][y-hsize][x+i] = 0
#
#     pil = topil(pic.squeeze(0).cpu())
#     pil = transforms.Resize(size=ori_size, interpolation=Image.BICUBIC)(pil)
#
#     pil.save(f"box/{list_selected[j]+1}.jpg")


In [ ]:
# hotbox

# for j in range(8):
#     pic = torch.zeros(1,3,512,512)
#     size = patch_sizes[j]
#     points = patch_centers[j]
#
#     hsize = size//2
#
#     for y,x in points:
#         pic[0,0,y-hsize:y+hsize+1,x-hsize:x+hsize+1] +=1
#
#
#     pic = pic[0,0].numpy()
#     fig,_ = plt.subplots()
#     plt.imshow(pic)
#     plt.tight_layout()
#     plt.show()
#     fig.savefig(f'./hotbox/{list_selected[j]+1}.jpg')


